In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

commitnumber = 523cbb7
molecule = {'co2': 0.0008}
band = [7]
atmpro = trp
tsfc = 300


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band5
  atmpro = mls
  band = 5
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.5e-24
  molecule = co2
  ng_adju = [0, 0]
  ng_refs = [1, 2]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (500, 250)]
  tsfc = 294
  vmax = 1100
  vmin = 980
  w_diffuse = [(1.75,), (1.66, 1.9)]
  wgt = [(0.75,), (0.75, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312        1 -0.001375                    1 -0.001257
0.000750        2 -0.001378                    2 -0.001249
0.001052        3 -0.001382                    3 -0.001274
0.001476        4 -0.001385                    4 -0.001279
0.002070        5 -0.001387                    5 -0.001284
0.002904        6 -0.001387                    6 -0.001286
0.004074        7 -0.001386                    7 -0.001287
0.005714        8 -0.001383                    8 -0.001287
0.008015        9 -0.001379                    9 -0.001284
0.011243       10 -0.001373                   10 -0.001285
0.015771       11 -0.001365                   11 -0.001291
0.022122       12 -0.001356                   12 -0.001294
0.031031       13 -0.001346                   13 -0.001293
0.043528       14 -0.001328                   14 -0.001285
0.061057       15 -0.001235                   15 -0.001239
0.085645       16 -0.000857                   16 -0.001019
0.120136       17  0.000044                   17 -0.000443
0.168516       18  0.001694                   18  0.000652
0.236378       19  0.004458                   19  0.002509
0.331549       20  0.008827                   20  0.005467
0.465100       21  0.015450                   21  0.010101
0.652400       22  0.024041                   22  0.016587
0.915100       23  0.027017                   23  0.019334
1.283650       24  0.022561                   24  0.016651
1.800600       25  0.016463                   25  0.012491
2.525700       26  0.010335                   26  0.008019
3.542800       27  0.005800                   27  0.004484
4.969550       28  0.002772                   28  0.001960
6.970850       29  0.000804                   29  0.000191
9.778100       30 -0.000442                   30 -0.001010
13.715850      31 -0.001205                   31 -0.001784
19.239350      32 -0.001649                   32 -0.002237
26.987250      33 -0.001882                   33 -0.002438
37.855300      34 -0.001973                   34 -0.002420
53.100050      35 -0.001905                   35 -0.002279
73.887500      36 -0.001648                   36 -0.002028
97.662500      37 -0.001490                   37 -0.001859
121.437500     38 -0.001736                   38 -0.002122
145.212500     39 -0.002239                   39 -0.002627
168.987500     40 -0.002705                   40 -0.003071
192.762500     41 -0.003107                   41 -0.003530
216.537500     42 -0.003426                   42 -0.004017
240.312500     43 -0.003643                   43 -0.004375
264.087500     44 -0.003748                   44 -0.004564
287.862500     45 -0.003733                   45 -0.004602
311.637500     46 -0.003588                   46 -0.004501
335.412500     47 -0.003309                   47 -0.004203
359.187500     48 -0.002890                   48 -0.003774
382.962500     49 -0.002327                   49 -0.003206
406.737500     50 -0.001617                   50 -0.002460
430.512500     51 -0.000757                   51 -0.001540
454.287500     52  0.000254                   52 -0.000466
478.062500     53  0.001419                   53  0.000785
501.837500     54  0.002740                   54  0.002210
525.612500     55  0.004218                   55  0.003793
549.387500     56  0.005855                   56  0.005543
573.162500     57  0.007654                   57  0.007482
596.937500     58  0.009616                   58  0.009605
620.712500     59  0.011736                   59  0.011915
644.487500     60  0.014002                   60  0.014347
668.262500     61  0.016363                   61  0.016855
692.037500     62  0.018695                   62  0.019371
715.812500     63  0.020816                   63  0.021730
739.587500     64  0.022638                   64  0.023865
763.362500     65  0.024281                   65  0.025861
78

# Fluxes by Layer

CRD                          CLIRAD             \
                        flug          fldg      fnetg      i       flug   
pressure    level                                                         
0.000000    1     -33.718068  0.000000e+00 -33.718068     11 -33.719989   
0.000624    2     -33.718068  5.552366e-08 -33.718068     11 -33.719989   
0.000876    3     -33.718068  7.837492e-08 -33.718068     11 -33.719989   
0.001229    4     -33.718068  1.118597e-07 -33.718068     11 -33.719989   
0.001723    5     -33.718068  1.609719e-07 -33.718068     11 -33.719989   
0.002417    6     -33.718068  2.329761e-07 -33.718068     11 -33.719989   
0.003391    7     -33.718068  3.385355e-07 -33.718068     11 -33.719990   
0.004757    8     -33.718069  4.932670e-07 -33.718068     11 -33.719990   
0.006672    9     -33.718069  7.200307e-07 -33.718069     11 -33.719990   
0.009359    10    -33.718070  1.052316e-06 -33.718069     11 -33.719991   
0.013128    11    -33.718071  1.539147e-06 -33.718070     11 -33.719992   
0.018415    12    -33.718073  2.252310e-06 -33.718071     11 -33.719993   
0.025830    13    -33.718075  3.296874e-06 -33.718072     11 -33.719995   
0.036232    14    -33.718078  4.826866e-06 -33.718073     11 -33.719998   
0.050823    15    -33.718083  7.094053e-06 -33.718076     11 -33.720002   
0.071291    16    -33.718090  1.093489e-05 -33.718079     11 -33.720008   
0.100000    17    -33.718100  1.871900e-05 -33.718082     11 -33.720017   
0.140271    18    -33.718117  3.527418e-05 -33.718081     11 -33.720031   
0.196760    19    -33.718140  7.005493e-05 -33.718070     11 -33.720052   
0.275997    20    -33.718170  1.420407e-04 -33.718028     11 -33.720080   
0.387100    21    -33.718201  2.885675e-04 -33.717912     11 -33.720113   
0.543100    22    -33.718208  5.818292e-04 -33.717627     11 -33.720139   
0.761700    23    -33.718140  1.136367e-03 -33.717004     11 -33.720132   
1.068500    24    -33.717985  1.962733e-03 -33.716022     11 -33.720092   
1.498800    25    -33.717808  2.936113e-03 -33.714872     11 -33.720079   
2.102400    26    -33.717666  3.970789e-03 -33.713695     11 -33.720146   
2.949000    27    -33.717629  4.970623e-03 -33.712659     11 -33.720362   
4.136600    28    -33.717749  5.906221e-03 -33.711843     11 -33.720780   
5.802500    29    -33.718076  6.780100e-03 -33.711295     11 -33.721462   
8.139200    30    -33.718677  7.603997e-03 -33.711073     11 -33.722484   
11.417000   31    -33.719642  8.397144e-03 -33.711245     11 -33.723944   
16.014700   32    -33.721083  9.182169e-03 -33.711901     11 -33.725962   
22.464000   33    -33.723141  9.980559e-03 -33.713160     11 -33.728687   
31.510500   34    -33.725984  1.080740e-02 -33.715177     11 -33.732278   
44.200100   35    -33.729800  1.165779e-02 -33.718142     11 -33.736835   
62.000000   36    -33.734544  1.238602e-02 -33.722158     11 -33.742427   
85.775000   37    -33.739673  1.287507e-02 -33.726798     11 -33.748720   
109.550000  38    -33.744216  1.322245e-02 -33.730994     11 -33.754377   
133.325000  39    -33.749679  1.379724e-02 -33.735882     11 -33.760967   
157.100000  40    -33.757214  1.502773e-02 -33.742187     11 -33.769525   
180.875000  41    -33.766993  1.718753e-02 -33.749805     11 -33.780072   
204.650000  42    -33.779118  2.056168e-02 -33.758556     11 -33.792931   
228.425000  43    -33.793648  2.544390e-02 -33.768204     11 -33.808559   
252.200000  44    -33.810591  3.212740e-02 -33.778463     11 -33.826897   
275.975000  45    -33.829927  4.090709e-02 -33.789020     11 -33.847728   
299.750000  46    -33.851608  5.207594e-02 -33.799532     11 -33.870910   
323.525000  47    -33.875560  6.592229e-02 -33.809638     11 -33.896382   
347.300000  48    -33.901686  8.272860e-02 -33.818957     11 -33.923731   
371.075000  49    -33.929867  1.027704e-01 -33.827097     11 -33.952958   
394.850000  50    -33.959962  1.263109e-01 -33.833651     11 -33.984070   
418.625000  51    -33.991805  1.536008e-01 -33.838

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')